In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
import numpy as np
from export.saver import read_pickle, save_pickle
from read.svg_reader import SVGReader
from helpers.helpers import filter_none
from placement2.attract import *
from visuals.plotter import plot_general
from copy import deepcopy
import networkx as nx
from itertools import pairwise, accumulate, product
from functools import reduce
from operator import add
from helpers.helpers import sort_and_group_objects
from cli.test import *

In [6]:
ureg = UnitRegistry()
inches = [1, 3/4]
fractional_inches = sum([Fraction(i) for i in inches])
inches_as_meters = (fractional_inches * ureg.inches).to(ureg.meters)
inches_as_meters.magnitude

Fraction(889, 20000)

In [10]:
feet = 2
feet_as_meters = (Fraction(feet)  * ureg.feet).to(ureg.meters)
res  = feet_as_meters + inches_as_meters
res

<Quantity(8175624999999999/12500000000000000, 'meter')>

In [14]:
frac = res.magnitude
with localcontext() as ctx:
        ctx.prec = 4
        res2 = frac.numerator / Decimal(frac.denominator)
res2

Decimal('0.6540')

In [17]:
case  = "amber_c_f01.svg"
sv = SVGReader(case)
sv.run()
# ag = AdjacencyGenerator(sv.layout, buffer_size=0.17)
# ag.run() # TODO shouldnt run if sv.layout is empty
# ag.draw_graph()
domains = sv.layout.domains
pos = create_pos(domains)


No reference for svg to meters conversion!! 


In [18]:
plot_general(sv.layout.domains)


In [19]:
x_ad = adjust_domains_x(domains)

In [20]:
y_ad = adjust_domains_y(x_ad)

In [21]:
create_ranges_for_all_nodes(domains, "y")

{'bed1_closet': {'bed1': [8.79-8.87]},
 'transit': {'m_bed': [8.79-8.87], 'bed1': [8.79-8.87]},
 'm_closet': {'m_bed': [8.82-8.87]},
 'laundry': {'bed1_closet': [8.34-8.42], 'bed1': [8.34-8.87]},
 'den': {'laundry': [7.17-7.25],
  'linen': [7.17-7.25],
  'transit': [7.17-7.83],
  'bed1_closet': [7.17-8.42],
  'bed1': [7.17-8.87]},
 'kitchen': {'bath': [5.27-5.43],
  'm_bath': [5.27-6.49],
  'm_closet': [5.27-7.94],
  'm_bed': [5.27-8.87]},
 'living': {'dining': [2.42-2.42],
  'kitchen': [2.42-2.60],
  'corridor': [2.42-5.02],
  'den': [2.42-5.12],
  'bath': [2.42-5.43],
  'm_bath': [2.42-6.49],
  'laundry': [2.42-7.25]},
 'dining': {'corridor': [5.02-5.02],
  'den': [5.02-5.12],
  'laundry': [5.02-7.25],
  'linen': [5.02-7.25],
  'transit': [5.02-7.83],
  'bed1_closet': [5.02-8.42],
  'm_bed': [5.02-8.87],
  'bed1': [5.02-8.87]},
 'den_closet': {'den': [5.02-5.12],
  'laundry': [5.02-7.25],
  'bed1_closet': [5.02-8.42],
  'bed1': [5.02-8.87]},
 'm_bath': {'m_closet': [7.73-7.94], 'm_be

In [7]:
domains

{'m_bed': Domain(m_bed, x=[0.11-2.24], y=[8.87-11.32]),
 'bed1_closet': Domain(bed1_closet, x=[3.03-4.18], y=[8.42-8.79]),
 'transit': Domain(transit, x=[1.79-2.94], y=[7.83-8.79]),
 'm_closet': Domain(m_closet, x=[0.11-1.58], y=[7.94-8.82]),
 'laundry': Domain(laundry, x=[3.03-4.18], y=[7.25-8.34]),
 'den': Domain(den, x=[2.37-4.18], y=[5.12-7.17]),
 'kitchen': Domain(kitchen, x=[0.11-1.58], y=[2.60-5.27]),
 'living': Domain(living, x=[0.11-3.27], y=[0.00-2.42]),
 'dining': Domain(dining, x=[1.73-3.27], y=[2.42-5.02]),
 'den_closet': Domain(den_closet, x=[3.38-4.18], y=[4.66-5.02]),
 'm_bath': Domain(m_bath, x=[0.00-1.61], y=[6.49-7.73]),
 'bath': Domain(bath, x=[0.03-1.61], y=[5.43-6.36]),
 'linen': Domain(linen, x=[2.37-2.94], y=[7.25-7.64]),
 'corridor': Domain(corridor, x=[1.79-2.24], y=[5.02-7.73]),
 'bed1': Domain(bed1, x=[2.37-4.18], y=[8.87-11.32])}

In [8]:
x_ad

{'m_bed': Domain(m_bed, x=[0.11-2.24], y=[8.87-11.32]),
 'bed1_closet': Domain(bed1_closet, x=[2.73-3.88], y=[8.42-8.79]),
 'transit': Domain(transit, x=[1.58-2.73], y=[7.83-8.79]),
 'm_closet': Domain(m_closet, x=[0.11-1.58], y=[7.94-8.82]),
 'laundry': Domain(laundry, x=[2.73-3.88], y=[7.25-8.34]),
 'den': Domain(den, x=[2.06-3.87], y=[5.12-7.17]),
 'kitchen': Domain(kitchen, x=[0.11-1.58], y=[2.60-5.27]),
 'living': Domain(living, x=[0.11-3.27], y=[0.00-2.42]),
 'dining': Domain(dining, x=[1.58-3.12], y=[2.42-5.02]),
 'den_closet': Domain(den_closet, x=[3.12-3.92], y=[4.66-5.02]),
 'm_bath': Domain(m_bath, x=[0.00-1.61], y=[6.49-7.73]),
 'bath': Domain(bath, x=[0.03-1.61], y=[5.43-6.36]),
 'linen': Domain(linen, x=[2.06-2.63], y=[7.25-7.64]),
 'corridor': Domain(corridor, x=[1.61-2.06], y=[5.02-7.73]),
 'bed1': Domain(bed1, x=[2.24-4.05], y=[8.87-11.32])}

In [9]:
y_ad

{'m_bed': Domain(m_bed, x=[0.11-2.24], y=[8.689999997999999999999999876-11.13999999799999999999999988]),
 'bed1_closet': Domain(bed1_closet, x=[2.73-3.88], y=[8.159999998999999999999999938-8.529999998999999999999999938]),
 'transit': Domain(transit, x=[1.58-2.73], y=[7.729999997999999999999999876-8.689999997999999999999999876]),
 'm_closet': Domain(m_closet, x=[0.11-1.58], y=[7.26-8.14]),
 'laundry': Domain(laundry, x=[2.73-3.88], y=[7.069999998999999999999999938-8.159999998999999999999999938]),
 'den': Domain(den, x=[2.06-3.87], y=[5.019999998999999999999999938-7.069999998999999999999999938]),
 'kitchen': Domain(kitchen, x=[0.11-1.58], y=[2.42-5.09]),
 'living': Domain(living, x=[0.11-3.27], y=[0.00-2.42]),
 'dining': Domain(dining, x=[1.58-3.12], y=[2.419999998999999999999999938-5.019999998999999999999999938]),
 'den_closet': Domain(den_closet, x=[3.12-3.92], y=[2.42-2.78]),
 'm_bath': Domain(m_bath, x=[0.00-1.61], y=[6.02-7.26]),
 'bath': Domain(bath, x=[0.03-1.61], y=[5.09-6.02]),


In [22]:
plot_general(y_ad)